In [1]:
import sys
import re
import time
import datetime
# Requires for ipython to pick up on twitter?
sys.path.append('/Library/Python/2.7/site-packages/')
import twitter
import pandas as pd
import func
# import pyowm # Historical API is paid

# inline plot
%matplotlib inline

/Users/albarron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
#%load 'data/raw-twt2016-01-26-14/21/09.csv'
truth = pd.read_csv("data/truth_tweets.csv",sep=',',error_bad_lines=False)
twts = pd.read_csv("data/formated_twts.csv",sep=',',error_bad_lines=False)
weather = pd.read_csv("data/weather-add-twt2016-03-06-21:26:39.csv",sep='\t',error_bad_lines=False)

In [3]:
del truth['Unnamed: 0']
truth.head()

,id,text,train_id,is_delay,delay_minor,delay_med,delay_major,delay_catastrophic,is_backlog,is_canceled,is_passing,is_accident,is_medical,is_mechanical,is_customer,is_event
0,691768068385718275,#NB155 is 22 mins late. #Caltrain,155,1,0,0,1,0,0,0,0,0,0,0,0,0
1,690667494906814464,#NB 151 is running 10 mins behind #Caltrain,151,1,1,0,0,0,0,0,0,0,0,0,0,0
2,689936168893329408,#NB151 is 10 mins down at San Mateo. #Caltrain,151,1,1,0,0,0,0,0,0,0,0,0,0,0
3,689509733640732672,#SB138 will board on the northbound platform a...,138,0,0,0,0,0,0,0,0,0,0,0,0,0
4,689485484838416388,#SB216 delayed 13 minutes at SAT\n#NB225 delay...,216,1,0,1,0,0,0,0,0,0,0,0,0,0


In [4]:
del twts['Unnamed: 0']
twts.head(3)

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,retweet_count,text,day_of_week,day_of_month,month,time_of_day,topic_train,t_bullet,t_limited,t_northbound,timestamp,train_id,tweet_id
0,2016-01-25 23:42:14,1,"NB155, Caltrain",691768068385718275,NaN,2,#NB155 is 22 mins late. #Caltrain,0,25,1,error,'155',0,0,1,2016-01-25 23:42:14,155,691768068385718275
1,2016-01-22 22:48:57,NaN,"NB, Caltrain",690667494906814464,NaN,2,#NB 151 is running 10 mins behind #Caltrain,4,22,1,error,'151',0,0,1,2016-01-22 22:48:57,151,690667494906814464
2,2016-01-20 22:22:55,1,"NB151, Caltrain",689936168893329408,NaN,2,#NB151 is 10 mins down at San Mateo. #Caltrain,2,20,1,error,'151',0,0,1,2016-01-20 22:22:55,151,689936168893329408


In [5]:
truth['uid'] = truth.apply(lambda x: int(str(x.train_id)+str(x.id)),axis=1)
twts['uid'] = truth.apply(lambda x: int(str(x.train_id)+str(x.id)),axis=1)

In [6]:
# df = twts.merge(truth,on=['id','train_id'],how='inner') # now have uid
df = twts.merge(truth,on='uid',how='inner')
df.shape

(584, 35)

In [7]:
wt = weather[['id','temp','precipiation','visability','windspeed','humidity','cloudcover']]

In [11]:
wt.head(2)

,id,temp,precipiation,visability,windspeed,humidity,cloudcover
0,691768068385718275,52.890417,0.000000,10.00,3.979583,0.811667,0.1555
1,690667494906814464,59.498750,0.003975,8.58,9.688750,0.820833,0.2325


In [13]:
df = df.merge(wt,left_on='tweet_id',right_on='id',how='left')
df.shape

(891, 42)

In [17]:
df.columns.values

array(['created_at', 'favorite_count', 'hashtags', 'id_x',
       'in_reply_to_screen_name', 'retweet_count', 'text_x', 'day_of_week',
       'day_of_month', 'month', 'time_of_day', 'topic_train', 't_bullet',
       't_limited', 't_northbound', 'timestamp', 'train_id_x', 'tweet_id',
       'uid', 'id_y', 'text_y', 'train_id_y', 'is_delay', 'delay_minor',
       'delay_med', 'delay_major', 'delay_catastrophic', 'is_backlog',
       'is_canceled', 'is_passing', 'is_accident', 'is_medical',
       'is_mechanical', 'is_customer', 'is_event', 'id', 'temp',
       'precipiation', 'visability', 'windspeed', 'humidity', 'cloudcover'], dtype=object)

In [18]:
del df['created_at']
del df['favorite_count']
del df['hashtags']
del df['in_reply_to_screen_name']
del df['retweet_count']
del df['text_y']
del df['train_id_y']
del df['id_y']
del df['id_x']

In [24]:
df.corr()

,day_of_week,day_of_month,month,t_bullet,t_limited,t_northbound,train_id_x,tweet_id,delay_minor,delay_med,...,is_mechanical,is_customer,is_event,id,temp,precipiation,visability,windspeed,humidity,cloudcover
day_of_week,1.000000,-0.115785,-0.065140,0.035467,-0.018641,-0.149511,0.131766,-0.152950,-0.087721,-0.106118,...,-0.004179,-0.076104,-0.046657,-0.152950,0.033963,-0.199958,0.150204,-0.088273,0.116645,0.044224
day_of_month,-0.115785,1.000000,-0.215266,-0.066065,-0.020689,0.232345,-0.088080,-0.233638,0.183974,0.184291,...,0.047268,0.058776,0.031172,-0.233638,0.028541,-0.066113,-0.027848,0.134544,-0.215640,-0.441157
month,-0.065140,-0.215266,1.000000,0.005550,0.124226,0.042219,0.146525,-0.148828,-0.018267,-0.012861,...,0.014185,0.012067,0.003991,-0.148828,0.296113,-0.102507,0.351216,0.013239,-0.185253,-0.060328
t_bullet,0.035467,-0.066065,0.005550,1.000000,-0.476382,-0.052987,0.687771,-0.054075,-0.055257,0.003468,...,0.058876,-0.016736,-0.035551,-0.054075,0.066375,-0.021347,0.051397,0.082620,0.096350,0.068440
t_limited,-0.018641,-0.020689,0.124226,-0.476382,1.000000,-0.060476,0.071475,-0.126792,-0.035435,-0.020007,...,-0.015265,-0.081034,-0.017438,-0.126792,0.194217,-0.016609,0.077472,0.131791,-0.096310,0.037573
t_northbound,-0.149511,0.232345,0.042219,-0.052987,-0.060476,1.000000,-0.051730,-0.030996,0.094855,0.048811,...,-0.036608,-0.002472,0.021182,-0.030996,0.067304,0.005826,-0.073545,0.089519,-0.181319,-0.190434
train_id_x,0.131766,-0.088080,0.146525,0.687771,0.071475,-0.051730,1.000000,-0.176209,-0.111687,-0.033601,...,0.057466,-0.082977,-0.021232,-0.176209,0.223346,-0.077480,-0.007836,0.201534,0.072768,0.029658
tweet_id,-0.152950,-0.233638,-0.148828,-0.054075,-0.126792,-0.030996,-0.176209,1.000000,-0.083385,-0.106140,...,-0.187601,0.011012,-0.061131,1.000000,-0.690081,0.234067,-0.347874,-0.625910,0.364412,0.199438
delay_minor,-0.087721,0.183974,-0.018267,-0.055257,-0.035435,0.094855,-0.111687,-0.083385,1.000000,-0.218760,...,-0.055742,0.045830,0.104024,-0.083385,0.030079,0.002867,0.004669,0.132641,-0.120101,-0.155543
delay_med,-0.106118,0.184291,-0.012861,0.003468,-0.020007,0.048811,-0.033601,-0.106140,-0.218760,1.000000,...,0.204219,0.228774,0.082463,-0.106140,0.023838,0.046172,-0.019883,0.071075,-0.171387,-0.173179


In [28]:
del df['text_x']

KeyError: 'text_x'

In [29]:
filename = "./data/merged_delay_{date}.csv".format(date=datetime.datetime.now().strftime("%Y-%m-%d-%H:%M:%S"))
df.to_csv(filename, sep='\t', encoding='utf-8')